2.ipynb：一份PDF文档，有三个（问题，答案）对 - 进阶RAG（检索器优化） - Ragas评估

- 3.2.ipynb：优化后的检索器 + 基于BGE Reranker的自定义文本压缩器 = 上下文压缩

# 加载环境变量

In [1]:
import os
from dotenv import find_dotenv, load_dotenv

# 加载环境变量
load_dotenv(find_dotenv())

True

# 源数据

[2023全球智能汽车AI挑战赛——赛道一：AI大模型检索问答](https://tianchi.aliyun.com/competition/entrance/532154/forum)

# 构建RAG

In [2]:
def pretty_print_docs(docs):
  print(
      f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n + {d.page_content}\n{d.metadata}" for i,d in enumerate(docs)])
  )

In [3]:
from langchain_community.document_loaders import PyPDFLoader

# 加载文档
file_path = "data/初赛训练数据集.pdf"
loader = PyPDFLoader(file_path)
docs = loader.load()
print(len(docs))

354


In [4]:
pretty_print_docs(docs[:3])

Document 1:

 + 欢迎
感谢您选择了具有优良安全性、舒适性、动力性和经济性的Lynk&Co领克汽车。
首次使用前请仔细、完整地阅读本手册内容，将有助于您更好地了解和使用车辆。
本手册中的所有资料均为出版时的最新资料，但本公司将对产品进行不断的改进和优化，您所购的车辆可能与本手册中的描述有所不同，请以实际
接收的车辆为准。
如您有任何问题，或需要预约服务，请拨打电话4006-010101 联系我们。您也可以开车前往Lynk &Co领克中心。
在抵达之前，请您注意驾车安全。©领克汽车销售有限公司
{'source': 'data/初赛训练数据集.pdf', 'page': 0}
----------------------------------------------------------------------------------------------------
Document 2:

 + 
{'source': 'data/初赛训练数据集.pdf', 'page': 1}
----------------------------------------------------------------------------------------------------
Document 3:

 + 3目录
前言
本手册相关的重要信息 .................................................11
敬告用户.................................................................11
联系Lynk&Co领克 .....................................................12
事件数据记录系统 ......................................................12
远程监控系统............................................................12
原厂精装附件、选装装备和改装 ......................................13
无线电设备 ..................

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 分割文档
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=256,
    chunk_overlap=50,
)
split_docs = text_splitter.split_documents(docs)
print(len(split_docs))

807


In [6]:
pretty_print_docs(split_docs[:3])

Document 1:

 + 欢迎
感谢您选择了具有优良安全性、舒适性、动力性和经济性的Lynk&Co领克汽车。
首次使用前请仔细、完整地阅读本手册内容，将有助于您更好地了解和使用车辆。
本手册中的所有资料均为出版时的最新资料，但本公司将对产品进行不断的改进和优化，您所购的车辆可能与本手册中的描述有所不同，请以实际
接收的车辆为准。
如您有任何问题，或需要预约服务，请拨打电话4006-010101 联系我们。您也可以开车前往Lynk &Co领克中心。
在抵达之前，请您注意驾车安全。©领克汽车销售有限公司
{'source': 'data/初赛训练数据集.pdf', 'page': 0}
----------------------------------------------------------------------------------------------------
Document 2:

 + 3目录
前言
本手册相关的重要信息 .................................................11
敬告用户.................................................................11
联系Lynk&Co领克 .....................................................12
{'source': 'data/初赛训练数据集.pdf', 'page': 2}
----------------------------------------------------------------------------------------------------
Document 3:

 + 事件数据记录系统 ......................................................12
远程监控系统............................................................12
原厂精装附件、选装装备和改装 ......................................13
{'source': 'data/初赛训练数据集.

In [7]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

# 创建嵌入模型
model_name = 'BAAI/bge-large-zh-v1.5'
model_kwargs = {'device': 'cuda'}  # 需要安装GPU版本的torch 
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

D:\Software\anaconda3\envs\py38_AdvancedRAG\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from langchain_community.vectorstores import FAISS

# 创建向量存储
vectordb = FAISS.from_documents(split_docs, embeddings)

In [9]:
index_folder_path = "data/faiss_index"
index_name = "3.2"

In [10]:
# 保存索引
vectordb.save_local(index_folder_path, index_name)

In [11]:
# 加载索引
vectordb = FAISS.load_local(index_folder_path, embeddings, index_name)

In [12]:
# 创建密集检索器，擅长根据语义相似度查找相关文档
faiss_retriever = vectordb.as_retriever(search_kwargs={"k": 10})

In [13]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

# 创建稀疏检索器，擅长根据关键词查找相关文档
bm25_retriever = BM25Retriever.from_documents(split_docs)
bm25_retriever.k=10

# 创建混合检索器
retriever = EnsembleRetriever(retrievers=[bm25_retriever, faiss_retriever], 
                              weight=[0.5, 0.5])

测试

In [14]:
question = "怎么打开危险警告灯？"
retrieval_docs = retriever.invoke(question)
len(retrieval_docs)

20

In [15]:
# 去重后
len(list(set([doc.page_content for doc in retrieval_docs])))

20

In [16]:
pretty_print_docs(retrieval_docs)

Document 1:

 + 技术资料
354公制术语
术语 说明
% 百分比
X:1 比值
℃ 摄氏温度
Ah 安时
m 米
cm 厘米
mm 毫米
g 克
kg 千克
h 小时
min 分钟
s 秒
rpm 每分钟转数
km/h 千米每小时
L 升
mL 毫升术语 说明
N 牛
Nm 牛米
V 伏特
W 瓦特
kPa 千帕
kW 千瓦
{'source': 'data/初赛训练数据集.pdf', 'page': 353}
----------------------------------------------------------------------------------------------------
Document 2:

 + 使用危险警告灯 ........................................................ 89
{'source': 'data/初赛训练数据集.pdf', 'page': 3}
----------------------------------------------------------------------------------------------------
Document 3:

 + 安全出行
110打开/关闭全景天窗
全景天窗由两部分组成，您可以使用车顶控制面板上的控制按钮，起
翘/滑动前半部分。后半部分为固定部分。
全景天窗配备有遮阳帘，位于车顶玻璃下方，在阳光强烈的情况下提
供保护。
将点火开关转至 I或以上挡位，可以操作全景天窗和遮阳帘。
打开/关闭全景天窗
手动滑动打开（轻按按钮至第1个停止位置）。
自动滑动打开（按到底）。
手动滑动关闭（轻按按钮至第1个停止位置）。
自动滑动关闭（按到底）。
如果全景天窗和遮阳帘处于完全关闭状态，轻按控制按钮，先打开遮
{'source': 'data/初赛训练数据集.pdf', 'page': 109}
----------------------------------------------------------------------------------------------------
Document 4:

 + 指示灯闪烁。打开危险警告灯开关时，两侧转
向指示灯均闪烁。
近光灯指

In [17]:
pretty_print_docs(faiss_retriever.invoke(question))

Document 1:

 + 使用危险警告灯 ........................................................ 89
{'source': 'data/初赛训练数据集.pdf', 'page': 3}
----------------------------------------------------------------------------------------------------
Document 2:

 + 指示灯闪烁。打开危险警告灯开关时，两侧转
向指示灯均闪烁。
近光灯指示灯：打开近光灯时，该指示灯点亮。
远光灯指示灯：打开远光灯时，该指示灯点亮。
智能远近光控制开启指示灯：打开智能远近光控制时，该
指示灯点亮。
后雾灯指示灯：打开后雾灯时，该指示灯点亮。
位置灯指示灯：打开位置灯时，该指示灯点亮。
泊车紧急制动指示灯：泊车紧急制动激活时，该指示灯点
亮。
{'source': 'data/初赛训练数据集.pdf', 'page': 73}
----------------------------------------------------------------------------------------------------
Document 3:

 + 灯点亮。
陡坡缓降系统故障警告灯：陡坡缓降系统出现故障时，该
警告灯点亮。
安全气囊故障警告灯：安全气囊系统或预紧器系统存在故
障时，该警告灯点亮。
{'source': 'data/初赛训练数据集.pdf', 'page': 72}
----------------------------------------------------------------------------------------------------
Document 4:

 + 仪表和灯光
89
01点击开启/关闭门控灯。
使用超车灯
超车时，您可将拨杆朝向自身方向拉动然后松开，此时远光灯闪烁一
次，以提醒前方车辆。
使用危险警告灯
危险警告灯按键
车辆遇到交通事故或其他紧急情况时，按下危险警告灯按键，启用危
险警告灯。
说明！
□发生碰撞情况下，危险警告灯也将自动点亮。
{'source': 'data/初赛训练数据集.pdf', '

In [18]:
pretty_print_docs(bm25_retriever.invoke(question))

Document 1:

 + 技术资料
354公制术语
术语 说明
% 百分比
X:1 比值
℃ 摄氏温度
Ah 安时
m 米
cm 厘米
mm 毫米
g 克
kg 千克
h 小时
min 分钟
s 秒
rpm 每分钟转数
km/h 千米每小时
L 升
mL 毫升术语 说明
N 牛
Nm 牛米
V 伏特
W 瓦特
kPa 千帕
kW 千瓦
{'source': 'data/初赛训练数据集.pdf', 'page': 353}
----------------------------------------------------------------------------------------------------
Document 2:

 + 安全出行
110打开/关闭全景天窗
全景天窗由两部分组成，您可以使用车顶控制面板上的控制按钮，起
翘/滑动前半部分。后半部分为固定部分。
全景天窗配备有遮阳帘，位于车顶玻璃下方，在阳光强烈的情况下提
供保护。
将点火开关转至 I或以上挡位，可以操作全景天窗和遮阳帘。
打开/关闭全景天窗
手动滑动打开（轻按按钮至第1个停止位置）。
自动滑动打开（按到底）。
手动滑动关闭（轻按按钮至第1个停止位置）。
自动滑动关闭（按到底）。
如果全景天窗和遮阳帘处于完全关闭状态，轻按控制按钮，先打开遮
{'source': 'data/初赛训练数据集.pdf', 'page': 109}
----------------------------------------------------------------------------------------------------
Document 3:

 + 安全出行
114
2点击
、
等指示箭头，调节选择的座椅功能。
前排座椅加热
通过中央显示屏调节
设计前排座椅加热功能的目的在于通过加热前排座椅，提高环境舒适
度。
您可以通过中央显示屏，设置驾驶员/副驾驶员侧座椅加热强度或关
闭座椅加热功能。
在中央显示屏中点击
 ，进入驾驶员侧座椅加热控制界面。
01设置驾驶员侧座椅加热强度及开关控制。
在中央显示屏中点击
 ，进入副驾驶员侧座椅加热控制界面。
{'source': 'data/初赛训练数据集.pdf', 'page': 113}
----------------

In [19]:
from langchain_openai import ChatOpenAI

# 创建模型
llm = ChatOpenAI(temperature=0)
print(llm.model_name)

gpt-3.5-turbo


# 重排序

## 思考过程：如何自定义文本压缩器

之前使用的是 LLMChainExtractor 

现在打算使用重排序模型做上下文压缩

参考 coherererank 怎么做的

[CohereRerank教程](https://python.langchain.com/docs/integrations/retrievers/cohere-reranker#doing-reranking-with-coherererank)

[CohereRerank文档](https://api.python.langchain.com/en/v0.0.354/retrievers/langchain.retrievers.document_compressors.cohere_rerank.CohereRerank.html#langchain.retrievers.document_compressors.cohere_rerank.CohereRerank)

[CohereRerank代码](https://github.com/langchain-ai/langchain/blob/v0.0.354/libs/langchain/langchain/retrievers/document_compressors/cohere_rerank.py)

```python
from langchain.retrievers.document_compressors import CohereRerank
compressor = CohereRerank()
```

我们需要构建一个与 CohereRerank 类相似的类，这个类使用我们自己的reranker模型

CohereRerank 类代码：

```python
from __future__ import annotations

from typing import TYPE_CHECKING, Dict, Optional, Sequence

from langchain_core.documents import Document
from langchain_core.pydantic_v1 import Extra, root_validator

from langchain.callbacks.manager import Callbacks
from langchain.retrievers.document_compressors.base import BaseDocumentCompressor
from langchain.utils import get_from_dict_or_env

if TYPE_CHECKING:
    from cohere import Client
else:
    # We do to avoid pydantic annotation issues when actually instantiating
    # while keeping this import optional
    try:
        from cohere import Client
    except ImportError:
        pass


class CohereRerank(BaseDocumentCompressor):
    """Document compressor that uses `Cohere Rerank API`."""

    client: Client
    """Cohere client to use for compressing documents."""
    top_n: int = 3
    """Number of documents to return."""
    model: str = "rerank-english-v2.0"
    """Model to use for reranking."""

    cohere_api_key: Optional[str] = None
    user_agent: str = "langchain"
    """Identifier for the application making the request."""

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
        arbitrary_types_allowed = True

    @root_validator(pre=True)
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that api key and python package exists in environment."""
        try:
            import cohere
        except ImportError:
            raise ImportError(
                "Could not import cohere python package. "
                "Please install it with `pip install cohere`."
            )
        cohere_api_key = get_from_dict_or_env(
            values, "cohere_api_key", "COHERE_API_KEY"
        )
        client_name = values.get("user_agent", "langchain")
        values["client"] = cohere.Client(cohere_api_key, client_name=client_name)
        return values

    def compress_documents(
        self,
        documents: Sequence[Document],
        query: str,
        callbacks: Optional[Callbacks] = None,
    ) -> Sequence[Document]:
        """
        Compress documents using Cohere's rerank API.

        Args:
            documents: A sequence of documents to compress.
            query: The query to use for compressing the documents.
            callbacks: Callbacks to run during the compression process.

        Returns:
            A sequence of compressed documents.
        """
        if len(documents) == 0:  # to avoid empty api call
            return []
        doc_list = list(documents)
        _docs = [d.page_content for d in doc_list]
        results = self.client.rerank(
            model=self.model, query=query, documents=_docs, top_n=self.top_n
        )
        final_results = []
        for r in results:
            doc = doc_list[r.index]
            doc.metadata["relevance_score"] = r.relevance_score
            final_results.append(doc)
        return final_results
```

基础架构类似：

```python
from langchain.retrievers.document_compressors.base import BaseDocumentCompressor

class BgeRerank(BaseDocumentCompressor):
    # 这里放一些类属性
    
    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
        arbitrary_types_allowed = True
        
    def compress_documents(
        self,
        documents: Sequence[Document],
        query: str,
        callbacks: Optional[Callbacks] = None,
    ) -> Sequence[Document]:
        """
        Compress documents using BAAI/bge-reranker models.

        Args:
            documents: A sequence of documents to compress.
            query: The query to use for compressing the documents.
            callbacks: Callbacks to run during the compression process.

        Returns:
            A sequence of compressed documents.
        """
        if len(documents) == 0:  # to avoid empty api call
            return []
        doc_list = list(documents)
        _docs = [d.page_content for d in doc_list]
        results = # self.client.rerank(
            # model=self.model, query=query, documents=_docs, top_n=self.top_n
        # )
        final_results = []
        for r in results:
            doc = # doc_list[r.index]
            doc.metadata["relevance_score"] = # r.relevance_score
            final_results.append(doc)
        return final_results
```

重点就是如何构建compress_documents函数，其实大部分都是差不多的，只有这部分是cohere rerank 特有的


```python
results = self.client.rerank(
            model=self.model, query=query, documents=_docs, top_n=self.top_n
        )
```

传入的是query, _docs, top_n

query是文本字符串

_docs是文本字符串列表

最终想要的是\[Document\]列表

bge-reranker-large 模型的使用方式：

```python
from sentence_transformers import CrossEncoder

model_name = 'BAAI/bge-reranker-large'
model = CrossEncoder(model_name)
pairs = [['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']]
model.predict(pairs)
# array([0.00365301, 0.9968658 ], dtype=float32)
```

```python
# 伪代码

# 待排序的对
pairs = [[问题，上下文], [问题，上下文]]
pairs = [[query，上下文], [query，上下文]]
pairs = []
for doc in _docs:
    pairs.append([query, doc])
pairs = [[query, doc] for doc in _docs]

# 模型推理
scores = model.predict(pairs)
scores = array([0.00365301, 0.9968658 ]  # len(_docs)个项

# 把scores的数值从高到低排序
results = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)
results = [(1, tensor(5.7623)), (0, tensor(-5.6085))]  # len(_docs)个项
return results[:top_n]  # top_n个项
```

写出排序函数 bge_rerank

```python
def bge_rerank(self, query, documents, top_n):
        pairs = [[query, doc] for doc in documents]
        scores = self.model.predict(pairs)
        print(scores)
        results = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)  # 得分从高到低排
        return results[:top_n]
```

调用bge_rerank函数：

```python
results = self.bge_rerank(query=query, documents=_docs, top_n=self.top_n)
```

得到最终的results，形如：

```python
results = [(1, tensor(5.7623)), (0, tensor(-5.6085))]  # top_n个项
```

后续处理：

```python
results =self.bge_rerank(query=query, documents=_docs, top_n=self.top_n)

final_results = []
    for r in results:
        doc = doc_list[r[0]]  # 次序为r[0]时的document（Document对象）
        doc.metadata["relevance_score"] = r[1] # 当前pair[query, doc.page_content]的相关度得分
        final_results.append(doc)
    return final_results
```

输出：
final_results 是top_n个Document 类对象组成的列表：[Document, Document]

## 实现：创建BgeRerank类

In [20]:
from __future__ import annotations

from typing import Dict, Optional, Sequence

from langchain_core.documents import Document
from langchain_core.pydantic_v1 import Extra, root_validator

from langchain.callbacks.manager import Callbacks
from langchain.retrievers.document_compressors.base import BaseDocumentCompressor

from sentence_transformers import CrossEncoder


class BgeRerank(BaseDocumentCompressor):
    """Document compressor that uses `BAAI/bge-reranker-large`."""

    top_n: int = 3
    """Number of documents to return."""
    model_name: str = "BAAI/bge-reranker-large"
    """Model name to use for reranking."""
    model:CrossEncoder = CrossEncoder(model_name)
    
    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
        arbitrary_types_allowed = True
        
    def bge_rerank(self, query, documents, top_n):

        pairs = [[query, doc] for doc in documents]
        scores = self.model.predict(pairs)
        print(scores)
        results = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)  # 得分从高到低排
        return results[:top_n]
    
    def compress_documents(
        self,
        documents: Sequence[Document],
        query: str,
        callbacks: Optional[Callbacks] = None,
    ) -> Sequence[Document]:
        """
        Compress documents using BAAI/bge-reranker-large.

        Args:
            documents: A sequence of documents to compress.
            query: The query to use for compressing the documents.
            callbacks: Callbacks to run during the compression process.

        Returns:
            A sequence of compressed documents.
        """
        if len(documents) == 0:  # to avoid empty api call
            return []
        # print(documents[0])
        doc_list = list(documents)
        # print(doc_list[0])
        _docs = [d.page_content for d in doc_list]
        # print(_docs[0])
        results = self.bge_rerank(query=query, documents=_docs, top_n=self.top_n)
        # print(results)
        final_results = []
        for r in results:
            # print(r[0])
            doc = doc_list[r[0]]
            # print(doc)
            doc.metadata["relevance_score"] = r[1]
            final_results.append(doc)
        return final_results

In [21]:
# 创建文档压缩器
compressor = BgeRerank()
compressor

BgeRerank(top_n=3, model_name='BAAI/bge-reranker-large', model=<sentence_transformers.cross_encoder.CrossEncoder.CrossEncoder object at 0x000002442DDE3DC0>)

In [22]:
question, len(retrieval_docs)

('怎么打开危险警告灯？', 20)

In [23]:
compressor.compress_documents(retrieval_docs, question)

[7.6233002e-05 5.6584144e-01 8.2255222e-02 9.8321229e-01 8.0388138e-04
 1.9083141e-03 9.9169648e-01 4.9066180e-03 1.1478431e-04 2.4551088e-02
 2.7098062e-02 3.0554244e-01 3.2997616e-02 3.1665735e-02 5.6071650e-02
 5.2805473e-03 8.4226400e-02 1.0856920e-02 6.5101929e-02 7.4016303e-01]


[Document(page_content='仪表和灯光\n89\n01点击开启/关闭门控灯。\n使用超车灯\n超车时，您可将拨杆朝向自身方向拉动然后松开，此时远光灯闪烁一\n次，以提醒前方车辆。\n使用危险警告灯\n危险警告灯按键\n车辆遇到交通事故或其他紧急情况时，按下危险警告灯按键，启用危\n险警告灯。\n说明！\n□发生碰撞情况下，危险警告灯也将自动点亮。', metadata={'source': 'data/初赛训练数据集.pdf', 'page': 88, 'relevance_score': 0.9916965}),
 Document(page_content='指示灯闪烁。打开危险警告灯开关时，两侧转\n向指示灯均闪烁。\n近光灯指示灯：打开近光灯时，该指示灯点亮。\n远光灯指示灯：打开远光灯时，该指示灯点亮。\n智能远近光控制开启指示灯：打开智能远近光控制时，该\n指示灯点亮。\n后雾灯指示灯：打开后雾灯时，该指示灯点亮。\n位置灯指示灯：打开位置灯时，该指示灯点亮。\n泊车紧急制动指示灯：泊车紧急制动激活时，该指示灯点\n亮。', metadata={'source': 'data/初赛训练数据集.pdf', 'page': 73, 'relevance_score': 0.9832123}),
 Document(page_content='仪表和灯光\n74\n盲点监测故障警告灯：盲点监测系统出现故障时，该警告\n灯常亮。盲点监测系统标定未完成时，该警告灯闪烁。\n低速行人提示音功能故障警告灯：低速行人提示音功能故\n障时，该警告灯点亮。\n指示灯和警告灯\n指示灯图标\n信息/通知用黄色显示：此信息指示灯点亮，同时仪表显\n示屏上出现对应文本信息。该指示灯也可能会与其他图标\n一同点亮。\nREADY指示灯：车辆准备就绪时，该指示灯点亮。\n转向指示灯：拨动转向指示灯开关时，对应的\n指示灯闪烁。打开危险警告灯开关时，两侧转\n向指示灯均闪烁。', metadata={'source': 'data/初赛训练数据集.pdf', 'page': 73, 'relevance_score': 0.740163})]

In [24]:
pretty_print_docs(compressor.compress_documents(retrieval_docs, question))

[7.6233002e-05 5.6584144e-01 8.2255222e-02 9.8321229e-01 8.0388138e-04
 1.9083141e-03 9.9169648e-01 4.9066180e-03 1.1478431e-04 2.4551088e-02
 2.7098062e-02 3.0554244e-01 3.2997616e-02 3.1665735e-02 5.6071650e-02
 5.2805473e-03 8.4226400e-02 1.0856920e-02 6.5101929e-02 7.4016303e-01]
Document 1:

 + 仪表和灯光
89
01点击开启/关闭门控灯。
使用超车灯
超车时，您可将拨杆朝向自身方向拉动然后松开，此时远光灯闪烁一
次，以提醒前方车辆。
使用危险警告灯
危险警告灯按键
车辆遇到交通事故或其他紧急情况时，按下危险警告灯按键，启用危
险警告灯。
说明！
□发生碰撞情况下，危险警告灯也将自动点亮。
{'source': 'data/初赛训练数据集.pdf', 'page': 88, 'relevance_score': 0.9916965}
----------------------------------------------------------------------------------------------------
Document 2:

 + 指示灯闪烁。打开危险警告灯开关时，两侧转
向指示灯均闪烁。
近光灯指示灯：打开近光灯时，该指示灯点亮。
远光灯指示灯：打开远光灯时，该指示灯点亮。
智能远近光控制开启指示灯：打开智能远近光控制时，该
指示灯点亮。
后雾灯指示灯：打开后雾灯时，该指示灯点亮。
位置灯指示灯：打开位置灯时，该指示灯点亮。
泊车紧急制动指示灯：泊车紧急制动激活时，该指示灯点
亮。
{'source': 'data/初赛训练数据集.pdf', 'page': 73, 'relevance_score': 0.9832123}
----------------------------------------------------------------------------------------------------
Docum

In [25]:
from langchain.retrievers import ContextualCompressionRetriever

# 创建上下文压缩检索器：需要传入一个文档压缩器和基本检索器
# 上下文压缩检索器将查询传递到基本检索器，获取初始文档并将它们传递到文档压缩器。
# 文档压缩器获取文档列表，并通过减少文档内容或完全删除文档来缩短文档列表。
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)


# 链

In [26]:
from langchain.chains import RetrievalQA

# 创建链
chain = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=compression_retriever,
                                 return_source_documents=True)

In [ ]:
# # 运行链 - 测试
# question = "怎么打开危险警告灯？"
# response = chain(question)
# print(response)

In [ ]:
# response['query']

In [ ]:
# response['result']

In [ ]:
# pretty_print_docs(response['source_documents'])

# 评估RAG

[使用您的测试集进行评估](https://docs.ragas.io/en/stable/getstarted/evaluation.html)

数据集包含以下列：

- question: list[str] - 这些是将评估您的 RAG 管道的问题。
- answer: list[str] - 您的 RAG 管道生成的答案。
- context: list[list[str]] - 传递到 LLM 来回答问题的上下文。
- ground_truth: list[str] - 问题的真实答案。


In [27]:
questions = ["怎么打开危险警告灯？",
            "车辆如何保养？",
            "靠背太热怎么办？"]

ground_truths = ["危险警告灯开关在方向盘下方，按下开关即可打开危险警告灯。",
            "为了保持车辆处于最佳状态，建议您定期关注车辆状态，包括定期保养、洗车、内部清洁、外部清洁、轮胎的保养、低压蓄电池的保养等。",
            "您好，如果您的座椅靠背太热，可以尝试关闭座椅加热功能。在多媒体显示屏上依次点击空调开启按键→座椅→加热，在该界面下可以关闭座椅加热。"]

In [28]:
# 生成 answers 和 contexts
answers = []
contexts = []

for question in questions:
    print(question)
    response = chain(question)
    print(response['result'], "\n")
    answers.append(response['result'])
    contexts.append([doc.page_content for doc in response['source_documents']])

怎么打开危险警告灯？
[7.6233002e-05 5.6584144e-01 8.2255222e-02 9.8321229e-01 8.0388138e-04
 1.9083141e-03 9.9169648e-01 4.9066180e-03 1.1478431e-04 2.4551088e-02
 2.7098062e-02 3.0554244e-01 3.2997616e-02 3.1665735e-02 5.6071650e-02
 5.2805473e-03 8.4226400e-02 1.0856920e-02 6.5101929e-02 7.4016303e-01]
要打开危险警告灯，您需要按下车辆上的危险警告灯按键。当车辆遇到交通事故或其他紧急情况时，按下该按键即可启用危险警告灯。在发生碰撞情况下，危险警告灯也会自动点亮。同时，打开危险警告灯开关时，两侧转向指示灯会同时闪烁，以提醒其他车辆。 

车辆如何保养？
[9.1936708e-05 9.2420524e-01 1.6697962e-02 4.3594980e-01 5.9080720e-01
 1.2467477e-02 3.0634513e-02 9.5204568e-01 6.4760244e-01 1.2674117e-02
 7.7452701e-01 1.2725201e-01 3.4151733e-02 7.4358690e-01 8.6873835e-01
 7.6462187e-05 6.3671267e-01 6.1603566e-03 1.4898573e-02 9.2054266e-01]
根据提供的信息，车辆保养包括以下几个方面：
1. 发动机保养：更换发动机机油和机油滤芯、空气滤芯、火花塞等。
2. 正时皮带保养：更换正时皮带、张紧器和惰轮。
3. 冷却液保养：定期更换冷却液。
4. 自动变速器油保养：更换自动变速器油。
5. 外部清洁：注意使用正确的清洁剂，避免使用含酸、强碱性、强化学性的清洁剂，以及避免使用含磨料的抛光剂。
6. 防腐蚀保养：定期检查车辆的防腐蚀保护，清洗和打蜡车辆，修复车漆小损伤，清洗车底的泥沙、污垢或盐积累。
7. 内饰保养：使用推荐的清洁剂和保养产品，定期用真空吸尘器清理内饰。

如果需要额外的保养项目，建议根据车辆的检查情况，向维修技

In [29]:
# 构建评估数据集
from datasets import Dataset

evaluate_data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": ground_truths
}

evaluate_dataset = Dataset.from_dict(evaluate_data)
evaluate_dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 3
})

In [30]:
evaluate_dataset[0]

{'question': '怎么打开危险警告灯？',
 'answer': '要打开危险警告灯，您需要按下车辆上的危险警告灯按键。当车辆遇到交通事故或其他紧急情况时，按下该按键即可启用危险警告灯。在发生碰撞情况下，危险警告灯也会自动点亮。同时，打开危险警告灯开关时，两侧转向指示灯会同时闪烁，以提醒其他车辆。',
 'contexts': ['仪表和灯光\n89\n01点击开启/关闭门控灯。\n使用超车灯\n超车时，您可将拨杆朝向自身方向拉动然后松开，此时远光灯闪烁一\n次，以提醒前方车辆。\n使用危险警告灯\n危险警告灯按键\n车辆遇到交通事故或其他紧急情况时，按下危险警告灯按键，启用危\n险警告灯。\n说明！\n□发生碰撞情况下，危险警告灯也将自动点亮。',
  '指示灯闪烁。打开危险警告灯开关时，两侧转\n向指示灯均闪烁。\n近光灯指示灯：打开近光灯时，该指示灯点亮。\n远光灯指示灯：打开远光灯时，该指示灯点亮。\n智能远近光控制开启指示灯：打开智能远近光控制时，该\n指示灯点亮。\n后雾灯指示灯：打开后雾灯时，该指示灯点亮。\n位置灯指示灯：打开位置灯时，该指示灯点亮。\n泊车紧急制动指示灯：泊车紧急制动激活时，该指示灯点\n亮。',
  '仪表和灯光\n74\n盲点监测故障警告灯：盲点监测系统出现故障时，该警告\n灯常亮。盲点监测系统标定未完成时，该警告灯闪烁。\n低速行人提示音功能故障警告灯：低速行人提示音功能故\n障时，该警告灯点亮。\n指示灯和警告灯\n指示灯图标\n信息/通知用黄色显示：此信息指示灯点亮，同时仪表显\n示屏上出现对应文本信息。该指示灯也可能会与其他图标\n一同点亮。\nREADY指示灯：车辆准备就绪时，该指示灯点亮。\n转向指示灯：拨动转向指示灯开关时，对应的\n指示灯闪烁。打开危险警告灯开关时，两侧转\n向指示灯均闪烁。'],
 'ground_truth': '危险警告灯开关在方向盘下方，按下开关即可打开危险警告灯。'}


[评估指标](https://docs.ragas.io/en/stable/concepts/metrics/index.html)
- Faithfulness
  - 如果答案中提出的所有主张都可以从给定的上下文中推断出来，则生成的答案被认为是忠实的。
  - answer 和 contexts
- Answer Relevancy
  - 评估指标“答案相关性”重点评估生成的答案与给定提示的相关程度。不完整或包含冗余信息的答案将获得较低分数。
  - answer 和 question
- Context Precision
  - 用于评估 contexts 中存在的所有真实相关项目是否排名较高。理想情况下，所有相关块必须出现在顶层。
  - question 和 contexts
- Context Recall
  - 上下文回忆衡量检索到的上下文与带注释的答案（被视为基本事实）的一致程度。
  - ground truth 和 contexts
---

- RAGAS评估框架评估了RAG管道的两个主要组件：
    - Retriever 检索器
    - Generator 生成器

---

- 与评估 Retrieval 相关的指标如下：
    - Context Precision。question 和 contexts
    - Context Recall。ground truth 和 contexts
- 与评估 Generation 相关的指标如下：
  - Faithfulness。answer 和 contexts
  - Answer Relevancy。answer 和 question

In [31]:
# 导入指标
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)


# 开始评估
from ragas import evaluate

evaluate_result = evaluate(
    evaluate_dataset, 
    metrics=[
        faithfulness,
        answer_relevancy,
        context_recall,
        context_precision,
    ]
)
evaluate_result

Evaluating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:37<00:00,  3.12s/it]


{'faithfulness': 1.0000, 'answer_relevancy': 0.9258, 'context_recall': 1.0000, 'context_precision': 0.5278}

In [32]:
df_evaluate_result = evaluate_result.to_pandas()
df_evaluate_result

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision
0,怎么打开危险警告灯？,要打开危险警告灯，您需要按下车辆上的危险警告灯按键。当车辆遇到交通事故或其他紧急情况时，按下...,[仪表和灯光\n89\n01点击开启/关闭门控灯。\n使用超车灯\n超车时，您可将拨杆朝向自...,危险警告灯开关在方向盘下方，按下开关即可打开危险警告灯。,1.0,0.992296,1.0,1.000000
1,车辆如何保养？,根据提供的信息，车辆保养包括以下几个方面：\n1. 发动机保养：更换发动机机油和机油滤芯、空...,[保养和维护\n321下表列举了您的爱车所需的常规保养项目。此外，您的维修技师可能\n会根据...,为了保持车辆处于最佳状态，建议您定期关注车辆状态，包括定期保养、洗车、内部清洁、外部清洁、轮...,1.0,0.939282,1.0,0.583333
2,靠背太热怎么办？,如果您觉得座椅靠背太热，可以尝试将座椅加热强度调至低档或者关闭座椅加热功能。根据提供的信息，...,[安全出行\n115\n01设置副驾驶员侧座椅加热强度及开关控制。\n驾驶员/副驾驶员侧座椅...,您好，如果您的座椅靠背太热，可以尝试关闭座椅加热功能。在多媒体显示屏上依次点击空调开启按键→...,1.0,0.845710,1.0,0.000000


In [33]:
df_evaluate_result.to_excel(f'data/{index_name}_eval_result.xlsx', index=False)